In [1]:
import json
import warnings
from pathlib import Path
import pickle

import numpy as np
import pandas as pd
import itertools
import seaborn as sns
from numpy import linalg as LA
import csv
sns.set_theme(style="darkgrid")

In [2]:
# current numpy and pandas versions have a FutureWarning for a mask operation we use;
# we will ignore it
warnings.filterwarnings("ignore", category=FutureWarning)

ROOT_DIRECTORY = Path("./")
DATA_DIRECTORY = ROOT_DIRECTORY

DEFAULT_SUBMISSION_FORMAT = DATA_DIRECTORY / "submission_format.csv"
DEFAULT_INPUT = DATA_DIRECTORY / "ground_truth.csv"
DEFAULT_PARAMS = DATA_DIRECTORY / "parameters.json"
DEFAULT_OUTPUT = ROOT_DIRECTORY / "submission.csv"

In [3]:
submission_format = DEFAULT_SUBMISSION_FORMAT
input_csv = DEFAULT_INPUT
params_file = DEFAULT_PARAMS

params = json.loads(params_file.read_text())
ipums = pd.read_csv(input_csv)

In [ ]:
# we want to understand how many users appear once, twice, ...
appearances = ipums['sim_individual_id'].value_counts()

sns.displot(appearances)

appearances = appearances.values

np.histogram(appearances, bins=range(1,21))

In [4]:
# bin the dataset
BINS = {
    "AGE": np.r_[-np.inf, np.arange(20, 105, 5), np.inf],
    "INCTOT": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCWAGE": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCWELFR": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCINVST": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCEARN": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "POVERTY": np.r_[-np.inf, np.arange(0, 520, 20), np.inf],
    "HHWT": np.r_[-np.inf, np.arange(0, 520, 20), np.inf],
    "PERWT": np.r_[-np.inf, np.arange(0, 520, 20), np.inf],
    "DEPARTS": np.r_[-np.inf, [h * 100 + m for h in range(24) for m in (0, 15, 30, 45)], np.inf],
    "ARRIVES": np.r_[-np.inf, [h * 100 + m for h in range(24) for m in (0, 15, 30, 45)], np.inf],
}

bins_for_numeric_cols = BINS
for col, bins in BINS.items():
    ipums[col] = pd.cut(ipums[col], bins).cat.codes

ipums.to_csv('binned_ground_truth.csv', index=False)


In [ ]:
# print domain size of each attribute
with params_file.open("r") as fp:
    parameters = json.load(fp)

attributes = columns = list(parameters["schema"].keys())

for att_i, att in enumerate(attributes):
    num_bins = len(BINS[att]) if att in BINS else len(params['schema'][att]['values'])
    print(att, num_bins)

In [ ]:
# distribution of singletons
single_attr_hist = {}
for attr in attributes:
    single_attr_hist[attr] = ipums[attr].value_counts()

In [ ]:
for attr in attributes:
    single_attr_hist[attr].to_csv(f'single_{attr}.csv')

In [ ]:
# distribution of pairs
pair_attr_hist = {}
for attr1, attr2 in itertools.combinations(attributes, 2):
    # print(attr1, attr2)
    groupby_df = ipums.groupby([attr1, attr2])
    indices = pd.MultiIndex.from_product([ipums[col].unique() for col in [attr1, attr2]])
    counts = groupby_df.size().to_frame('count').reindex(indices).fillna(0)

    pair_attr_hist[(attr1, attr2)] = counts


In [ ]:
total = 1033968

In [ ]:
# L1 error of guessing
# (by assuming independency of attributes and knowledge of singleton distrubtion)
pair_attr_hist_ind = {}
pair_attr_hist_l1 = np.zeros((len(attributes), len(attributes)))
for attr1_i, attr1 in enumerate(attributes):
    for attr2_i, attr2 in enumerate(attributes):
        if attr1_i >= attr2_i:
            continue

        ground_truth = pair_attr_hist[(attr1, attr2)].sort_index()['count'].values

        single1 = single_attr_hist[attr1].sort_index().values
        single2 = single_attr_hist[attr2].sort_index().values

        guess_ind = np.zeros_like(ground_truth)

        for i, v in enumerate(single1):
            guess_ind[i * len(single2) : (i + 1) * len(single2)] = v / total * single2

        pair_attr_hist_ind[(attr1, attr2)] = guess_ind
        pair_attr_hist_l1[attr1_i, attr2_i] = LA.norm(guess_ind - ground_truth, 1)

In [ ]:
np.savetxt("pair_l1.csv", pair_attr_hist_l1, delimiter=",")

In [ ]:
# save the l1 errors in a 1d array
pair_attr_hist_l1_1d = []
for attr1_i, attr1 in enumerate(attributes):
    for attr2_i, attr2 in enumerate(attributes):
        if attr1_i >= attr2_i:
            continue

        ground_truth = pair_attr_hist[(attr1, attr2)]['count'].values
        num_bins = 1
        for att in [attr1, attr2]:
            num_bins *= len(BINS[att]) if att in BINS else len(params['schema'][att]['values'])
        row = [attr1, attr2, num_bins, np.count_nonzero(ground_truth), pair_attr_hist_l1[attr1_i, attr2_i]]
        pair_attr_hist_l1_1d.append(row)

In [ ]:
with open("pair_l1_1d.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerows(pair_attr_hist_l1_1d)

In [ ]:
# understand l1 error of 3-way marginals fixing puma-year
# similar to the previous one but treating puma and year as a single attribute
puma_year_pair_attr_hist = {}
for attr in attributes:
    if attr in ['PUMA', 'YEAR']:
        continue
    # print(attr1, attr2)
    groupby_df = ipums.groupby(['PUMA', 'YEAR', attr])
    indices = pd.MultiIndex.from_product([ipums[col].unique() for col in ['PUMA', 'YEAR', attr]])
    counts = groupby_df.size().to_frame('count').reindex(indices).fillna(0)

    puma_year_pair_attr_hist[attr] = counts


In [ ]:
puma_year_pair_attr_hist_ind = {}
puma_year_pair_attr_hist_l1 = np.zeros((len(attributes), 3))
single1 = pair_attr_hist[('PUMA', 'YEAR')].sort_index()['count'].values

In [ ]:
for attr_i, attr in enumerate(attributes):

    if attr in ['PUMA', 'YEAR']:
        continue

    ground_truth = puma_year_pair_attr_hist[attr].sort_index()['count'].values
    num_bins = 181 * 7 * len(BINS[attr]) if attr in BINS else len(params['schema'][attr]['values'])

    single2 = single_attr_hist[attr].sort_index().values

    guess_ind = np.zeros_like(ground_truth)

    for i, v in enumerate(single1):
        guess_ind[i * len(single2) : (i + 1) * len(single2)] = v / total * single2

    puma_year_pair_attr_hist_ind[attr] = guess_ind
    puma_year_pair_attr_hist_l1[attr_i] = np.array([len(ground_truth), np.count_nonzero(ground_truth), LA.norm(guess_ind - ground_truth, 1)])

np.savetxt("puma_year_pair_l1.csv", puma_year_pair_attr_hist_l1, delimiter=",")

In [ ]:
# L1 error of guessing
# (by assuming independency of attributes and knowledge of singleton distribution)
pair_attr_hist_ind = {}
pair_attr_hist_l1 = np.zeros((len(attributes), len(attributes)))
for attr1_i, attr1 in enumerate(attributes):
    for attr2_i, attr2 in enumerate(attributes):
        if attr1_i >= attr2_i:
            continue

        ground_truth = pair_attr_hist[(attr1, attr2)].sort_index()['count'].values

        single1 = single_attr_hist[attr1].sort_index().values
        single2 = single_attr_hist[attr2].sort_index().values

        guess_ind = np.zeros_like(ground_truth)

        for i, v in enumerate(single1):
            guess_ind[i * len(single2) : (i + 1) * len(single2)] = v / total * single2

        pair_attr_hist_ind[(attr1, attr2)] = guess_ind
        pair_attr_hist_l1[attr1_i, attr2_i] = LA.norm(guess_ind - ground_truth, 1)

In [ ]:
np.savetxt("pair_l1.csv", pair_attr_hist_l1, delimiter=",")

In [ ]:
# understand the distance (l1 error of any pair) between the public data and any other state
import os
BINS = {
    "AGE": np.r_[-np.inf, np.arange(20, 105, 5), np.inf],
    "INCTOT": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCWAGE": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCWELFR": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCINVST": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "INCEARN": np.r_[-np.inf, np.arange(0, 105_000, 5_000), np.inf],
    "POVERTY": np.r_[-np.inf, np.arange(0, 520, 20), np.inf],
    "HHWT": np.r_[-np.inf, np.arange(0, 520, 20), np.inf],
    "PERWT": np.r_[-np.inf, np.arange(0, 520, 20), np.inf],
    "DEPARTS": np.r_[-np.inf, [h * 100 + m for h in range(24) for m in (0, 15, 30, 45)], np.inf],
    "ARRIVES": np.r_[-np.inf, [h * 100 + m for h in range(24) for m in (0, 15, 30, 45)], np.inf],
}

def bin_attr(cur_df):
    for col, bins in BINS.items():
        cur_df[col] = cur_df.cut(ipums[col], bins)
    return puma_df_i

def obtain_two_way_marginal(cur_df):
    cur_pair_attr_hist = {}
    for attr1, attr2 in itertools.combinations(attributes, 2):
        groupby_df = cur_df.groupby([attr1, attr2])
        indices = pd.MultiIndex.from_product([cur_df[col].unique() for col in [attr1, attr2]])
        counts = groupby_df.size().to_frame('count').reindex(indices).fillna(0)

        cur_pair_attr_hist[(attr1, attr2)] = counts
    return cur_pair_attr_hist

pub_path = f'ground_truth.csv'
pub_puma = pd.read_csv(pub_path)
for col, bins in BINS.items():
        pub_puma[col] = pd.cut(ipums[col], bins)

state_puma = {}
for i in range(55):
    path_i = f'{i}-data.csv'
    if os.path.isfile(path_i):
        puma_df_i = pd.read_csv(path_i)
        puma_df_i = bin_attr(puma_df_i)
        state_puma[i] = puma_df_i

pub_pair_attr_hist = obtain_two_way_marginal(pub_puma)
state_pair_attr_hist = {}
for state, state_df in state_puma.items():
    state_pair_attr_hist[state] = obtain_two_way_marginal(state_df)

attr_pairs = []
for attr1_i, attr1 in enumerate(attributes):
    if attr1 == 'PUMA':
        continue
    for attr2_i, attr2 in enumerate(attributes):
        if attr2 == 'PUMA' or attr1_i >= attr2_i:
            continue
        attr_pairs.append(tuple([attr1, attr2]))

output = np.array((len(state_puma), len(attr_pairs)))
for state_i, state in enumerate(state_puma.keys()):
    for attr_pair_i, attr_pair in enumerate(attr_pairs):
        output[state_i][attr_pair_i] = LA.norm(pub_pair_attr_hist[attr_pair] - state_pair_attr_hist[state], 1)

with open("states_pair_l1.csv", 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerows(pair_attr_hist_l1_1d)